## Conception du Modèle CNN

1_ Définir l’architecture CNN (Conv2D + MaxPooling + Dropout + Dense) avec des paramètres optimaux.

2_ Choisir les meilleures fonctions d’activation pour les couches cachées et la couche de sortie.

**** pour les couches cachées :Relu
     formule : f(x)= max(0,x)
     on a choisi relu car 
     _ on active juste les neurones pertinents on prend les valeur positives et pour les valeurs negatives devient 0
     _ rapide a calculer : accelere l'apprentisssage
     _ evite les gradients tres petits 
**** pour la couche de sortie : Softmax
     _ transforme les sorties en probablités pour chaque classe
     _ la somme de toutes les probabilites =1
     _ideal pour classifiication multi-classes

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np

num_classes = len(np.unique(labels))  # à définir selon ton dataset

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(num_classes, activation='softmax')
])

3_ Compiler le modèle (ex. : Optimiseur Adam, fonction de perte categorical_crossentropy).

=> Analyse :  

pour optimiser notre modele on fait recours a optimiseur adm ainsi que la fonction de perte 
on commence par Categorical Crossentropy qui mesure l’erreur entre la prédiction du modèle et le vrai label.
puis optimiseur  Adam utilise cette erreur pour ajuster les poids du réseau et améliorer la prédiction.

**** Adam (optimiseur)

Quand il intervient ?

Pendant l’entraînement du modèle, après chaque batch d’images.

Son rôle : mettre à jour les poids des neurones pour que le modèle fasse moins d’erreurs sur tes images IRM.

Exemple :

Tu passes une image IRM d’un gliome dans ton CNN.

Le modèle prédit : [0.2, 0.1, 0.6, 0.1] (probabilités pour [glioma, meningioma, notumor, pituitary])

Le vrai label (one-hot) est [1, 0, 0, 0]

Adam regarde la différence entre la prédiction et la vérité → et ajuste tous les filtres/convolutions dans le réseau pour que la prochaine fois, le modèle prédit mieux.



**** Categorical Crossentropy (fonction de perte)

Quand elle intervient ?

Toujours pendant l’entraînement, juste avant qu’Adam mette à jour les poids.

Elle calcule combien la prédiction du modèle est mauvaise par rapport au vrai label.

Exemple avec ton brief :

Même image IRM : vrai label [1, 0, 0, 0]

Modèle prédit [0.2, 0.1, 0.6, 0.1]

Crossentropy calcule “perte = 1.609” (exemple numérique) → plus la prédiction est loin du vrai label, plus la perte est grande

Adam utilise cette perte pour corriger les poids → le modèle apprend à mieux classer les gliomes.


__ En résumé simple :

Tu donnes une image au modèle.

Le modèle prédit des probabilités.

Categorical Crossentropy mesure l’erreur de prédiction.

Adam ajuste les poids du CNN pour réduire cette erreur.

Répète pour toutes les images, epoch après epoch → le modèle s’améliore.

In [ ]:
# Compilation du modèle
model.compile(
    optimizer='adam',                 
    loss='categorical_crossentropy',  
    metrics=['accuracy']              
)

4_ Utiliser les fonctions model.summary() et plot_model() pour vérifier l’architecture du modèle.
**** model.summary() va afficher dans la console toutes les couches du modèle, leurs sorties et le nombre de paramètres.

plot_model() crée un fichier image (cnn_model.png) avec une représentation graphique du CNN. show_shapes=True permet de voir la taille des sorties de chaque couche.

In [ ]:
from tensorflow.keras.utils import plot_model

# Afficher un résumé du modèle
model.summary()

5_ Déterminer les hyperparamètres (taux d’apprentissage, nombre d’époques, taille de batch).

Les hyperparamètres définissent le comportement du processus d’apprentissage d’un modèle de deep learning. Ils influencent directement la vitesse de convergence, la stabilité et la performance finale du modèle.

1️⃣ Taux d’apprentissage (learning_rate)

Définition :
Le taux d’apprentissage contrôle l’amplitude des mises à jour appliquées aux poids du modèle après chaque itération.
Il s’agit du paramètre le plus critique dans l’optimisation.

Valeur typique : 0.001
(valeur par défaut de nombreux optimiseurs comme Adam)

Effets :

🔹 Trop élevé → l’algorithme saute le minimum et ne converge pas.

🔹 Trop faible → la convergence devient très lente ou le modèle reste bloqué dans un minimum local.

Recommandation :
Tester plusieurs valeurs dans une plage comme [0.0001, 0.005], ou utiliser un scheduler pour l’ajuster dynamiquement.


2️⃣ Nombre d’époques (epochs)

Définition :
Une époque correspond à un passage complet sur l’ensemble des données d’entraînement.
Le nombre d’époques détermine combien de fois le modèle revoit les données pendant l’apprentissage.

Valeur typique : 10 à 50, selon la taille du dataset et la complexité du modèle.

Effets :

🔹 Trop faible → sous-apprentissage (le modèle n’a pas encore appris les motifs).

🔹 Trop élevé → surapprentissage (le modèle mémorise le dataset et perd en généralisation).

Astuce :
Utiliser un callback comme EarlyStopping pour arrêter l’entraînement quand la performance sur la validation cesse d’augmenter.

3️⃣ Taille de batch (batch_size)

Définition :
Le batch size indique le nombre d’échantillons traités avant chaque mise à jour des poids.
Chaque lot (batch) est utilisé pour calculer le gradient moyen avant la mise à jour.

Valeur typique : 16, 32, ou 64 selon la mémoire GPU disponible.

Effets :

🔹 Batch petit → apprentissage plus bruité mais meilleure généralisation.

🔹 Batch grand → apprentissage plus stable mais besoin de plus de mémoire.

Recommandation :
Commencer avec 32, puis ajuster selon les ressources matérielles et la stabilité de la loss.

In [ ]:
# Hyperparamètres principaux
learning_rate = 0.001   # Taux d'apprentissage pour Adam
epochs = 20             # Nombre d'époques pour l'entraînement
batch_size = 32         # Nombre d'images traitées à chaque itératio

6_Utiliser la bibliothèque time pour mesurer la durée de l’entraînement.

La bibliothèque time permet de mesurer le temps écoulé entre le début et la fin de l’entraînement d’un modèle de machine learning ou deep learning.
C’est utile pour :

Évaluer les performances du modèle et du matériel (CPU/GPU)

Comparer la vitesse d’entraînement entre différents modèles ou configurations

In [ ]:
start_time = time.time()

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[checkpoint]
)

end_time = time.time()
training_time = end_time - start_time
print(f"Durée totale de l'entraînement : {training_time:.2f} secondes")
print(f"Durée totale en minute : {training_time/60:.2f} minutes")